In [23]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# Carrega o dataset Iris
iris = load_iris()
df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
df['target'] = iris.target

# Seleciona apenas duas features (sepal length e petal width)
df = df[['sepal length (cm)', 'petal width (cm)', 'target']]

# Filtra as classes Iris setosa e Iris versicolor
df = df[df['target'].isin([0, 1])]  # Classes 0 e 1

# Define as features reais (índices das colunas)
R = [0, 1] 

# Separa os dados em features (X) e classes (y)
X = df.iloc[:, R]  # Seleciona as features reais
y = df['target']

# Divide os dados em conjuntos de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cria o modelo de regressão logística
modelo = LogisticRegression()

# Treina o modelo com os dados de treino
modelo.fit(X_train, y_train)

# Imprime os pesos encontrados
print(f"Pesos: {modelo.coef_}")
print(f"Intercepto: {modelo.intercept_}")

# Avalia o modelo com os dados de teste
score = modelo.score(X_test, y_test)
print(f"Acurácia: {score:.2f}")

# Define os pesos do classificador linear
w1 = modelo.coef_[0][0]  # Peso para 'sepal length'
w2 = modelo.coef_[0][1]  # Peso para 'petal width'
wo = modelo.intercept_[0]  # Termo constante

def ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe):
    """
    Encontrar uma PI-explicação usando um algoritmo guloso.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        R: Limite de explicação.
        Idx: Índice atual na lista delta.
        Xpl: Conjunto de literais da explicação.
        classe: Classe atual sendo considerada.

    Returns:
        Tupla com o limite atualizado (R) e o índice atualizado (Idx).
    """
    # Verifica se Idx está dentro dos limites da lista antes de incrementá-lo
    if Idx + 1 < len(delta):
        # Se o peso da primeira feature for maior
        if abs(w1) > abs(w2):
            Idx = 0 # Escolhe a primeira feature como a mais importante
        else:
            Idx = 1 # Escolhe a segunda feature como a mais importante

        R -= delta[Idx]
        # Converte a tupla para string para que seja "hashable"
        Xpl.add(str((Idx, Vs[Idx])))  # Adiciona o literal à PI-explicação
        REPORTEXPLANATION(Xpl, classe, w1, w2)  # Imprime ou processa a PI-explicação atual
        return R, Idx
    else:
        # Se Idx estiver fora dos limites, interrompe o loop
        return R, Idx

def ALLEXPLANATIONS(Vs, delta, threshold, w1, w2):
    """
    Enumerar todas as PI-explicações usando backtracking, considerando todas as classes.

    Args:
        Vs: Valores da instância.
        delta: Lista ordenada de valores de delta.
        threshold: Limite de explicação.
        w1: Peso para a primeira feature.
        w2: Peso para a segunda feature.

    Returns:
        Lista de tuplas com a classe e a PI-explicação para cada instância.
    """
    pi_explicacoes = []  # Lista para armazenar as PI-explicações
    for classe in range(2):  # Itera sobre as duas classes (0 e 1)
        Xpl = set()  # Conjunto de literais da explicação
        Idx = 0
        R = 0
        # Adiciona a condição para verificar o limite de Idx
        while Idx >= 0 and Idx < len(delta): 
            R, Idx = ONEEXPLANATION(Vs, delta, R, Idx, Xpl, classe)
            # Ajuste para garantir que R não se torne negativo
            if R < 0:
                R = 0
            # Adiciona a PI-explicação à lista, apenas se não existir na lista
            if (classe, Xpl) not in pi_explicacoes:
                pi_explicacoes.append((classe, Xpl))  # Adiciona a PI-explicação à lista
            # Verifica se a PI-explicação já foi encontrada para essa classe, interrompe o loop para a classe atual
            if len(pi_explicacoes) > 1 and pi_explicacoes[-1] == pi_explicacoes[-2]:
                break
            Idx += 1 # Incrementa Idx após a chamada da função ONEEXPLANATION
    return pi_explicacoes

def REPORTEXPLANATION(Xpl, classe, w1, w2):
    """Imprime a PI-explicação."""
    print(f"Classe: {classe}")
    print(f"PI-explicação: {Xpl}")
    EXPLICAR_PI(Xpl, w1, w2, df)

def EXPLICAR_PI(Xpl, w1, w2, df):
    """Explica os elementos da PI-explicação."""
    for item in Xpl:
        idx, valores = eval(item)
        if idx == 0:
            print(f"  - Sepal Length ({w1}): {valores[0]} cm")
            print(f"      - Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")
        else:
            print(f"  - Petal Width ({w2}): {valores[1]} cm")
            print(f"      - Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")

        # Imprime os dados da outra feature (mesmo que ela não esteja na PI-explicação)
        if idx == 0:
            print(f"  - Petal Width ({w2}): {valores[1]} cm")
            print(f"      - Valor Mínimo para Petal Width: {df['petal width (cm)'].min()} cm")
        else:
            print(f"  - Sepal Length ({w1}): {valores[0]} cm")
            print(f"      - Valor Mínimo para Sepal Length: {df['sepal length (cm)'].min()} cm")



# Cria uma lista para armazenar os valores de delta
delta = []

# Percorre cada feature selecionada
for feature in df.columns[:-1]:  # Exclui a coluna 'target'
    # Calcula a diferença entre o valor máximo e o valor mínimo da feature
    delta_feature = df[feature].max() - df[feature].min()
    delta.append(delta_feature)

# Define o limite de explicação (threshold)
threshold = 0  # Use 0 como threshold

# Cria uma lista com os valores de features para cada instância
Vs = []
for index, row in df.iterrows():
    Vs.append(list(row[:-1]))  # Excluir a coluna 'target'

# Chama a função para enumerar todas as PI-explicações
pi_explicacoes = ALLEXPLANATIONS(Vs, delta, threshold, w1, w2)

# Cria um DataFrame com as PI-explicações
pi_explicacoes_df = pd.DataFrame(pi_explicacoes, columns=['Classe', 'PI-Explicação'])

# Imprime o DataFrame
print(pi_explicacoes_df)

Pesos: [[1.37515913 3.50507158]]
Intercepto: [-9.9889242]
Acurácia: 1.00
Classe: 0
PI-explicação: {'(1, [4.9, 0.2])'}
  - Petal Width (3.505071575134545): 0.2 cm
      - Valor Mínimo para Petal Width: 0.1 cm
  - Sepal Length (1.3751591278961899): 4.9 cm
      - Valor Mínimo para Sepal Length: 4.3 cm
Classe: 1
PI-explicação: {'(1, [4.9, 0.2])'}
  - Petal Width (3.505071575134545): 0.2 cm
      - Valor Mínimo para Petal Width: 0.1 cm
  - Sepal Length (1.3751591278961899): 4.9 cm
      - Valor Mínimo para Sepal Length: 4.3 cm
   Classe      PI-Explicação
0       0  {(1, [4.9, 0.2])}
1       1  {(1, [4.9, 0.2])}
